In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
from matplotlib.lines import Line2D
import geopandas as gpd
from sklearn.metrics.pairwise import haversine_distances
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import rasterio as rio
from rasterio.plot import show
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import Normalize

In [3]:
dir_top_maxent = "/Users/liting/Documents/GitHub/r_chagasM/output/pixel_buffer_off"
dir_top_rf = "/Users/liting/Documents/GitHub/r_chagasM/output/rf/pixel_buffer_off"
dir_diff = "/Users/liting/Documents/GitHub/r_chagasM/output/figures/final_plot/difference"
dir_saving_plot_path = '/Users/liting/Documents/GitHub/r_chagasM/output/figures/final_plot/plots'
sp_rarefied_name_list_full = ['T. sanguisuga','T. rubida',' T. recurva','T. protracta','T. mexicana','T. mazzottii','T. neotomae','T. longipennis',' P. lecticularia',' T. indictiva','T. gerstaeckeri','T. dimidiata']
label_list = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']
bug_name_list = ["sanguisuga","rubida","recurva","protracta","mexicana","mazzottii","neotomae","longipennis","lecticularia","indictiva","gerstaeckeri","dimidiata"]

In [4]:
old_blue = plt.get_cmap('Blues_r')

# Create a new colormap that goes from white to the 'Blues' colormap
newcolors_1 = old_blue(np.linspace(0, 1, 256))
newcolors_1[-1, :] = np.array([1, 1, 1, 1])  # Set the first color to white
new_blue = LinearSegmentedColormap.from_list('Custom Blues_r', newcolors_1)

old_red = plt.get_cmap('Reds')

# Create a new colormap that goes from white to the 'Blues' colormap
newcolors_2 = old_red(np.linspace(0, 1, 256))
newcolors_2[0, :] = np.array([1, 1, 1, 1])  # Set the first color to white
new_red = LinearSegmentedColormap.from_list('Custom Reds', newcolors_2)


In [8]:

# Enable LaTeX rendering in Matplotlib
plt.rcParams['text.usetex'] = True
this_fontsize = 44
fig, ax = plt.subplots(5,4, figsize=(50,30))
bug_name_list = ['San','Rub','Rec','Pro','Mex','Maz','Neo','Lon','Lec','Ind','Ger','Dim']
raster_save_list = ["historical_predict_allinput_","ssp126_predict_allinput_","ssp245_predict_allinput_","ssp370_predict_allinput_","ssp585_predict_allinput_"]
diff_raster_save_list = ["diff_historical.tif","diff_ssp126.tif","diff_ssp245.tif","diff_ssp370.tif","diff_ssp585.tif"]
for j in range(len(bug_name_list)):

    this_bug = bug_name_list[j]
    print(this_bug)
    for i in range(20):
        
        ax_0 = i // 4
        ax_1 = i % 4

        
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
        world.boundary.plot(ax=ax[ax_0,ax_1], color = 'slategrey',linewidth=1,figsize=(6,5))


        # maxent
        if ax_1 == 0:
            name_for_saving = dir_top_maxent + '/' + this_bug+"/result/output_raster/all_input/"+ raster_save_list[ax_0] + this_bug + '.tif'
            predict_plot_final = rio.open(name_for_saving)
            show(predict_plot_final,cmap='viridis',ax=ax[ax_0,ax_1],vmin=0, vmax=1)

        # rf
        if ax_1 == 1:
            name_for_saving = dir_top_rf + '/' + this_bug+"/result/output_raster/all_input/"+ raster_save_list[ax_0] + this_bug + '.tif'
            predict_plot_final = rio.open(name_for_saving)
            show(predict_plot_final,cmap='viridis',ax=ax[ax_0,ax_1],vmin=0, vmax=1)

        # difference + 
        if ax_1 == 2:
            name_for_saving = dir_diff + '/' + this_bug + '/'+ diff_raster_save_list[ax_0]
            predict_plot_final = rio.open(name_for_saving)
            show(predict_plot_final,cmap=new_red,ax=ax[ax_0,ax_1],vmin=0, vmax=1)

        # difference - 
        if ax_1 == 3:
            name_for_saving = dir_diff + '/' + this_bug + '/'+ diff_raster_save_list[ax_0]
            predict_plot_final = rio.open(name_for_saving)
            show(predict_plot_final,cmap=new_blue,ax=ax[ax_0,ax_1],vmin=-1, vmax=0)
        
        ax[ax_0,ax_1].set_aspect('equal')
        ax[ax_0,ax_1].set_xlim([-180,-52])
        ax[ax_0,ax_1].set_ylim([13,85])
        ax[ax_0,ax_1].tick_params(left = False, right = False , labelleft = False ,labelbottom = False, bottom = False)
        

    plt.gcf().text(0.08, 0.90, label_list[j] + ') ' + r'$\textbf{\it{%s}}$' % sp_rarefied_name_list_full[j], fontsize=this_fontsize,weight='bold')
    plt.gcf().text(0.08, 0.80, '1) Historical', fontsize=this_fontsize)
    plt.gcf().text(0.08, 0.65, '2) SSP1 RCP2.6', fontsize=this_fontsize)
    plt.gcf().text(0.08, 0.50, '3) SSP2 RCP4.5', fontsize=this_fontsize)
    plt.gcf().text(0.08, 0.35, '4) SSP3 RCP7.0', fontsize=this_fontsize)
    plt.gcf().text(0.08, 0.20, '5) SSP5 RCP8.5', fontsize=this_fontsize)
    plt.gcf().text(0.215, 0.90, 'a) MaxEnt ('+"$p_{m}$"+')', fontsize=this_fontsize)
    plt.gcf().text(0.367, 0.90, 'b) Random Forest ('+"$p_{r}$"+')', fontsize=this_fontsize)
    plt.gcf().text(0.505, 0.90, 'c) Positive Difference '+'$(p_{r}-p_{m})_+$', fontsize=this_fontsize)
    plt.gcf().text(0.67, 0.90, 'd) Negative Difference '+'$(p_{r}-p_{m})_-$', fontsize=this_fontsize)
    
    plt.subplots_adjust(hspace=0.015)
    plt.subplots_adjust(wspace=-0.015)
    plt.subplots_adjust(left=0.16,right=0.84)

    diff_file_path = dir_diff +  '/' + this_bug + '/'+ "/sum_results.csv"   
    diff_df = pd.read_csv(diff_file_path)
    plt.gcf().text(0.85, 0.90, r'$(\overline{p_{r}}-\overline{p_{m}})/\overline{p_{m}}$', fontsize=this_fontsize)
    plt.gcf().text(0.86, 0.80, '%.1f'%(100*(diff_df['sum_diff_raster_historical'])/diff_df['sum_maxent_raster_historical']) + '\%', fontsize=this_fontsize)
    plt.gcf().text(0.86, 0.65, '%.1f'%(100*(diff_df['sum_diff_raster_ssp126'])/diff_df['sum_maxent_raster_ssp126']) + '\%', fontsize=this_fontsize)
    plt.gcf().text(0.86, 0.50, '%.1f'%(100*(diff_df['sum_diff_raster_ssp245'])/diff_df['sum_maxent_raster_ssp245']) + '\%', fontsize=this_fontsize)
    plt.gcf().text(0.86, 0.35, '%.1f'%(100*(diff_df['sum_diff_raster_ssp370'])/diff_df['sum_maxent_raster_ssp370']) + '\%', fontsize=this_fontsize)
    plt.gcf().text(0.86, 0.20, '%.1f'%(100*(diff_df['sum_diff_raster_ssp585'])/diff_df['sum_maxent_raster_ssp585']) + '\%', fontsize=this_fontsize)
    
    # # subplot for colorbar
    ax2 =fig.add_axes([0.17,0.08,0.32,0.02])
    cb2 = mpl.colorbar.ColorbarBase(ax2, orientation='horizontal', cmap='viridis')
    cb2.set_label("$p_{m}$"+', '+"$p_{r}$", labelpad=0, size=this_fontsize-5)
    cb2.ax.tick_params(labelsize=this_fontsize-10)
    # # subplot for colorbar
    norm3 = Normalize(vmin=0, vmax=1)
    ax3 =fig.add_axes([0.515,0.08,0.14,0.02])
    cb3 = mpl.colorbar.ColorbarBase(ax3, orientation='horizontal', cmap=new_red,norm=norm3)
    cb3.set_label("$(p_{r}-p_{m})_+$", labelpad=0, size=this_fontsize-5)
    cb3.ax.tick_params(labelsize=this_fontsize-10)
    # # subplot for colorbar
    norm4 = Normalize(vmin=-1, vmax=0)
    ax4 =fig.add_axes([0.685,0.08,0.14,0.02])
    cb4 = mpl.colorbar.ColorbarBase(ax4, orientation='horizontal', cmap=new_blue,norm=norm4)
    cb4.set_label("$(p_{r}-p_{m})_-$", labelpad=0, size=this_fontsize-5)
    cb4.ax.tick_params(labelsize=this_fontsize-10)
    #'T. dimidiata','T. gerstaeckeri','T. protracta','T. sanguisuga'
    plt.savefig(dir_saving_plot_path+'/'+this_bug+'.png',dpi=100)

San


/var/folders/dx/4fv665g177x973sw3f557wx80000gq/T/ipykernel_35095/3583105979.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
/var/folders/dx/4fv665g177x973sw3f557wx80000gq/T/ipykernel_35095/3583105979.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
/var/folders/dx/4fv665g177x973sw3f557wx80000gq/T/ipykernel_35095/3583105979.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.natura

Rub


/var/folders/dx/4fv665g177x973sw3f557wx80000gq/T/ipykernel_35095/3583105979.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
/Users/liting/anaconda3/envs/analysis/lib/python3.11/site-packages/matplotlib/image.py:499: RuntimeWarning: overflow encountered in divide
  A_scaled /= ((a_max - a_min) / frac)
/var/folders/dx/4fv665g177x973sw3f557wx80000gq/T/ipykernel_35095/3583105979.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
/Users/liting/anaconda3/envs/analysis/lib/python3.11/site-packages/matplotli

Rec


...

Lon


/var/folders/dx/4fv665g177x973sw3f557wx80000gq/T/ipykernel_35095/3583105979.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
/Users/liting/anaconda3/envs/analysis/lib/python3.11/site-packages/matplotlib/image.py:499: RuntimeWarning: overflow encountered in divide
  A_scaled /= ((a_max - a_min) / frac)
/var/folders/dx/4fv665g177x973sw3f557wx80000gq/T/ipykernel_35095/3583105979.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
/Users/liting/anaconda3/envs/analysis/lib/python3.11/site-packages/matplotli

Lec


/var/folders/dx/4fv665g177x973sw3f557wx80000gq/T/ipykernel_35095/3583105979.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
/Users/liting/anaconda3/envs/analysis/lib/python3.11/site-packages/matplotlib/image.py:499: RuntimeWarning: overflow encountered in divide
  A_scaled /= ((a_max - a_min) / frac)
/var/folders/dx/4fv665g177x973sw3f557wx80000gq/T/ipykernel_35095/3583105979.py:18: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
/Users/liting/anaconda3/envs/analysis/lib/python3.11/site-packages/matplotli

KeyboardInterrupt: 